In [2]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv
load_dotenv()

from services.twilio import client
from services.cache import get_agent_metadata, get_all_agents
from services.chat.chat import similarity_search, get_embedding
from services.db.supabase_services import supabase_client
supabase = supabase_client()

from app.core.config import settings

from composio import Composio

# Initialize the Composio client
client = Composio()

In [ ]:
""" Get all connected apps and accounts """
""" store connection ID to each user_id in supabase """


# Get the list of integrations (connected apps)
integrations = client.integrations.get()
print("Connected Apps:")
for integration in integrations:
    print(f"• App: {integration.appName}")
    print(f"  ID: {integration.id}")

# Get the list of connected accounts (existing connections)
connected_accounts = client.connected_accounts.get()
print("\nExisting Connections:")
for account in connected_accounts:
    print(f"• ID: {account.id}")
    print(f"  App: {account.appName}")
    print(f"  Status: {account.status}")


In [ ]:
from services.composio import get_calendar_slots

get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv")

In [75]:
from composio import Composio

# Initialize the Composio client
composio_client = Composio()

# Create a new entity with a custom ID
new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


In [ ]:
""" checks if entity already exists """

existing_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
existing_connections = existing_entity.get_connections()

if existing_connections:
    print("Entity already exists with connections:", existing_connections)
else:
    print("Entity is newly created or has no connections")


# # Check for connected accounts for the new entity
# try:
#     connected_account = new_entity.get_connection(app="outlook")
#     print(f"Connected Outlook account: {connected_account}")
# except Exception as e:
#     print("No connected Slack account found")

# # Perform actions using the new entity
# actions = new_entity.get_tools(app_name="github")

existing_connections

### Outlook

In [ ]:
""" this function will be triggered in endpoint /composion/new_connection """
""" then to store the connection ID to the user_id in supabase """
""" frontend to have button to "generate auth link", response from this endpoint will be the redirect url """
### entity ID will be created for each user. 
""" {"app": "outlook", "connectedAccountId" : "xxx", "client_id" : "xxx" } """

"""Auth"""
from composio import ComposioToolSet, App
from composio import Composio
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

# Initialize the Composio client
composio_client = Composio()

# # Create a new entity with a custom ID
# new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXvww")

# request = entity.initiate_connection(App.OUTLOOK)
# ### background task save request.connectedAccountId to integrations table in supabase 
# ### then return the below redirect url to frontend
# print(f"Open this URL to authenticate: {request.redirectUrl}")


In [ ]:
tools = toolset.get_tools(actions=[Action.OUTLOOK_LIST_EVENTS])

tools

In [ ]:
tools = toolset.get_tools(apps=[App.OUTLOOK])

tools

In [62]:
openai_client = OpenAI()

today = datetime.now().strftime("%Y-%m-%d")
task = f"get calendar slots for next week, today is {today}"

# Update the tools to include search capability
tools = toolset.get_tools(actions=[
    Action.OUTLOOK_OUTLOOK_UPDATE_CALENDAR_EVENT])


# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = toolset.handle_tool_calls(search_response)
search_result


In [ ]:
search_result

### Shopify

In [7]:
"""Auth"""
""" shopify not working :("""

from composio import ComposioToolSet, Composio, App
from dotenv import load_dotenv
load_dotenv()

client = Composio()
toolset = ComposioToolSet()
entity = toolset.get_entity()
request = entity.initiate_connection(App.SHOPIFY, connected_account_params={"shop": "fdi1gh-xg"})

print(f"Open this URL to authenticate: {request.redirectUrl}")


[2024-11-27 01:04:00,016][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Open this URL to authenticate: https://fdi1gh-xg.myshopify.com/admin/oauth/authorize?client_id=7c7a26e7515b114c6f992a99318c0e4c&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=read_products%2Cwrite_products%2Cread_orders%2Cwrite_orders&access_mode=per-user&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiJjMWExZmM3YS0wYzBkLTQwMzctYjkwOC00ZjQ5NzA1YmI1ZjciLCJpbnRlZ3JhdGlvbklkIjoiOGVhMjdkZjQtNDYyYy00ODY0LThlZTMtMTk3NDExOGJhMmRkIiwiYXBwTmFtZSI6InNob3BpZnkiLCJjbGllbnRJbmZvIjp7InByb2plY3RJZCI6IjAxMjcxNDU5LTg2M2YtNDc3MC1hOGEzLTE2NmFjYjc4MWYzNyIsIm1lbWJlcklkIjoiMmJkMjYwNWQtMmI5OC00MGU4LWJjNTYtZjhkNjZhMTU2OWVjIn0sImlhdCI6MTczMjY2OTQ0MX0.nvalCqNugOwpx2cyZ4GRJV9p0OJ75_8KXjoq2JnIPXc&code_challenge=k__YnLG-iuiomi5-scC8f7aD3875krwWXiTkB1h0df8&code_challenge_method=S256


### Notion

In [ ]:
"""Auth"""

from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


In [ ]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)



In [ ]:
print(search_result)


### Google Calendar

In [1]:
from composio_openai import ComposioToolSet, Action
from composio import Composio
from openai import OpenAI
from datetime import datetime 

from dotenv import load_dotenv
load_dotenv()

client = Composio()
openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

GOOGLECALENDAR_CREATE_EVENT: 'Action'
GOOGLECALENDAR_DELETE_EVENT: 'Action'
GOOGLECALENDAR_DUPLICATE_CALENDAR: 'Action'
GOOGLECALENDAR_FIND_EVENT: 'Action'
GOOGLECALENDAR_FIND_FREE_SLOTS: 'Action'
GOOGLECALENDAR_GET_CALENDAR: 'Action'
GOOGLECALENDAR_GET_CURRENT_DATE_TIME: 'Action'
GOOGLECALENDAR_LIST_CALENDARS: 'Action'
GOOGLECALENDAR_PATCH_CALENDAR: 'Action'
GOOGLECALENDAR_QUICK_ADD: 'Action'
GOOGLECALENDAR_REMOVE_ATTENDEE: 'Action'
GOOGLECALENDAR_UPDATE_EVENT: 'Action'

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_LIST_CALENDARS])
# task = "find free slots for next week"

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_GET_CALENDAR])
# task = "fetch calendar for id michael@flowon.ai"

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_FIND_FREE_SLOTS])
today = datetime.now().strftime("%Y-%m-%d")

task = f"Search for free slots for the next 14 days for id michael@flowon.ai. Today's date is {today}."

response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)

result = composio_toolset.handle_tool_calls(response)

[2024-11-27 02:08:25,698][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this
[2024-11-27 02:08:29,331][INFO] Executing `GOOGLECALENDAR_FIND_FREE_SLOTS` with params={'time_min': '2024,11,27,00,00,00', 'time_max': '2024,12,11,23,59,59', 'items': ['michael@flowon.ai']} and metadata={} connected_account_id=None
[2024-11-27 02:08:31,030][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#freeBusy', 'timeMin': '2024-11-27T00:00:00.000Z', 'timeMax': '2024-12-11T23:59:59.000Z', 'calendars': {'michael@flowon.ai': {'busy': [{'start': '2024-11-27T12:00:00Z', 'end': '...


In [2]:
from services.composio import get_calendar_slots, find_free_slots

await get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv","outlook")



[2024-11-27 02:08:31,046][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Getting calendar slots..


[2024-11-27 02:08:33,630][INFO] Executing `GOOGLECALENDAR_FIND_FREE_SLOTS` with params={'time_min': '2024,11,27,02,08,00', 'time_max': '2024,12,11,02,08,00', 'items': ['michael@flowon.ai']} and metadata={} connected_account_id=None
[2024-11-27 02:08:35,024][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#freeBusy', 'timeMin': '2024-11-27T02:08:00.000Z', 'timeMax': '2024-12-11T02:08:00.000Z', 'calendars': {'michael@flowon.ai': {'busy': [{'start': '2024-11-27T12:00:00Z', 'end': '...


{'formatted': {'27th Wednesday November 2024': ['09:00 AM',
   '09:30 AM',
   '10:00 AM',
   '10:30 AM',
   '11:00 AM',
   '11:30 AM',
   '01:30 PM',
   '02:00 PM',
   '02:30 PM',
   '03:30 PM',
   '04:00 PM',
   '04:30 PM'],
  '28th Thursday November 2024': ['09:00 AM'],
  '29th Friday November 2024': ['09:00 AM',
   '09:30 AM',
   '10:00 AM',
   '10:30 AM',
   '11:00 AM',
   '11:30 AM',
   '12:00 PM',
   '12:30 PM',
   '01:00 PM',
   '01:30 PM',
   '02:00 PM',
   '02:30 PM',
   '03:00 PM',
   '03:30 PM',
   '04:00 PM',
   '04:30 PM'],
  '30th Saturday November 2024': ['09:00 AM', '09:30 AM', '10:00 AM']}}

In [4]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Create a 1 hour meeting event at 5:30PM tomorrow regarding OpenAI Integration"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


[2024-11-27 02:05:14,398][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'start_datetime': '2024-11-28T17:30:00', 'event_duration': '1h', 'summary': 'OpenAI Integration Meeting', 'create_meeting_room': True} and metadata={} connected_account_id=None
[2024-11-27 02:05:16,412][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3465346231906000"', 'id': 'b452do6bfcqo7qibjhnut5sfk0', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=YjQ1MmRvNmJmY3FvN3FpYmpo...


[{'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3465346231906000"', 'id': 'b452do6bfcqo7qibjhnut5sfk0', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=YjQ1MmRvNmJmY3FvN3FpYmpobnV0NXNmazAgbWljaGFlbEBmbG93b24uYWk', 'created': '2024-11-27T02:05:15.000Z', 'updated': '2024-11-27T02:05:15.953Z', 'summary': 'OpenAI Integration Meeting', 'creator': {'email': 'michael@flowon.ai', 'self': True}, 'organizer': {'email': 'michael@flowon.ai', 'self': True}, 'start': {'dateTime': '2024-11-28T17:30:00Z', 'timeZone': 'UTC'}, 'end': {'dateTime': '2024-11-28T18:30:00Z', 'timeZone': 'UTC'}, 'iCalUID': 'b452do6bfcqo7qibjhnut5sfk0@google.com', 'sequence': 0, 'hangoutLink': 'https://meet.google.com/zgt-jkfo-hmj', 'conferenceData': {'createRequest': {'requestId': '6006688a68e84e34960623ff564e5667', 'conferenceSolutionKey': {'type': 'hangoutsMeet'}, 'status': {'statusCode': 'success'}}, 'entryPoints': [{'entryPointType': 'video', 'uri': 'http